### Part 3.2 - Model 1: Multi-Input Keras neural model (latest changes on 08.03.2020)

#### Import standard libraries

In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from tabulate import tabulate
import re
import os

import random

# Module to serialize the content produced from the execution of the code

import pickle

# Module to monitor the progress of a python for loop

from tqdm import tqdm_notebook

# Module to manipulate text in python - NLTK package

import nltk
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

# Module to compute word vectorizers and compute the cosine distance

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_distances

import string
import itertools

# from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:70% !important; }</style>"))

#### Keras Text Classification (For creating the word embeddings)

In [ ]:
from nltk.stem import WordNetLemmatizer

from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

from time import time

#--------------------------------------------------------------

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras import layers
from tensorflow.keras import models

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

#---------------------------------------------------------------

%matplotlib inline
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from pylab import rcParams

import pydot
import pydotplus
import graphviz

from IPython.display import SVG
from tensorflow.keras.utils import model_to_dot

from tensorflow.keras.models import load_model
from tensorflow.keras.models import model_from_json
import json

# Import ML FLow
import mlflow.tensorflow
import mlflow.pyfunc
from tensorflow.keras import regularizers
import datetime

# Import TensorBoard
import tensorflow_docs as tfdocs
import tensorflow_docs.plots as tfplots
import tensorflow_docs.modeling as tfmodel
from tensorflow.keras import regularizers
# from tensorboard import default
# from tensorboard import program

import tensorflow_hub as hub
import bert
from bert import tokenization
from bert.tokenization import FullTokenizer

#Visualize Model

def visualize_model(model):
    return SVG(model_to_dot(model, show_shapes= True, show_layer_names=True, dpi=65).create(prog='dot', format='svg'))

from tensorflow.keras.utils import plot_model

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import hamming_loss, zero_one_loss, f1_score

from packaging import version

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

#### Import the data already tokenized and transformed from Part 3.1

* 80-20 split - Non-balanced data

In [ ]:
X_train_seq_actors=np.load(os.path.join(os.getcwd(), "80-20 split_non-balanced\\x_train_seq_actors_80-20_non-balanced_06032020.npy"))
X_train_seq_plot=np.load(os.path.join(os.getcwd(), "80-20 split_non-balanced\\x_train_seq_plot_80-20_non-balanced_06032020.npy"))
X_train_seq_features=np.load(os.path.join(os.getcwd(), "80-20 split_non-balanced\\x_train_seq_features_80-20_non-balanced_06032020.npy"))
X_train_seq_reviews=np.load(os.path.join(os.getcwd(), "80-20 split_non-balanced\\x_train_seq_reviews_80-20_non-balanced_06032020.npy"))

print("X_train data inputs have been loaded!\n")

X_test_seq_actors=np.load(os.path.join(os.getcwd(), "80-20 split_non-balanced\\x_test_seq_actors_80-20_non-balanced_06032020.npy"))
X_test_seq_plot=np.load(os.path.join(os.getcwd(), "80-20 split_non-balanced\\x_test_seq_plot_80-20_non-balanced_06032020.npy"))
X_test_seq_features=np.load(os.path.join(os.getcwd(), "80-20 split_non-balanced\\x_test_seq_features_80-20_non-balanced_06032020.npy"))
X_test_seq_reviews=np.load(os.path.join(os.getcwd(), "80-20 split_non-balanced\\x_test_seq_reviews_80-20_non-balanced_06032020.npy"))

print("X_test data inputs have been loaded!\n")

y_train=np.load(os.path.join(os.getcwd(), "80-20 split_non-balanced\\y_train_80-20_non-balanced_06032020.npy"))
y_test=np.load(os.path.join(os.getcwd(), "80-20 split_non-balanced\\y_test_80-20_non-balanced_06032020.npy"))

print("y_train & y_test have been loaded!\n")

#### Import the saved tokenizers

In [ ]:
"""
IMport the tokenizers of each input, fitted on part 3.1
"""
with open(os.path.join(os.getcwd(), '80-20 split_non-balanced\\actors_tokenizer_06032020.pkl'),'rb') as f:
    actors_tokenizer = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '80-20 split_non-balanced\\plot_tokenizer_06032020.pkl'),'rb') as f:
    plot_tokenizer = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '80-20 split_non-balanced\\features_tokenizer_06032020.pkl'),'rb') as f:
    features_tokenizer = pickle.load(f)
    
with open(os.path.join(os.getcwd(), '80-20 split_non-balanced\\reviews_tokenizer_06032020.pkl'),'rb') as f:
    reviews_tokenizer = pickle.load(f)
    
print("Tokenizers are loaded successfully!")

In [ ]:
"""
KFold cross validation

Maybe I could run a Kfold for with k=1 to reshuffle the data.
"""
X_seq_actors=np.load("C:\\Users\\spano\\Desktop\\GitHub-Thesis\\models_text_classification\\80-20 split_non-balanced\\x_seq_actors_80-20_non-balanced_28022020.npy")
X_seq_plot=np.load("C:\\Users\\spano\\Desktop\\GitHub-Thesis\\models_text_classification\\80-20 split_non-balanced\\x_seq_plot_80-20_non-balanced_28022020.npy")
X_seq_features=np.load("C:\\Users\\spano\\Desktop\\GitHub-Thesis\\models_text_classification\\80-20 split_non-balanced\\x_seq_features_80-20_non-balanced_28022020.npy")
X_seq_reviews=np.load("C:\\Users\\spano\\Desktop\\GitHub-Thesis\\models_text_classification\\80-20 split_non-balanced\\x_seq_reviews_80-20_non-balanced_28022020.npy")

print("X featurs data inputs have been loaded!\n")

y=np.load("C:\\Users\\spano\\Desktop\\GitHub-Thesis\\models_text_classification\\80-20 split_non-balanced\\y_80-20_non-balanced_28022020.npy")

print("y variable has been loaded!\n")

print("X_seq_actors shape:{}".format(X_seq_actors.shape))
print("X_seq_plot shape:{}".format(X_seq_plot.shape))
print("X_seq_features shape:{}".format(X_seq_features.shape))
print("X_seq_reviews shape:{}\n".format(X_seq_reviews.shape))

print("y shape:{}".format(y.shape))

# <b>- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -  </b>

#### ML-FLOW approach (contrary to TensorBoard) - <i> 15.01.2020 </i>

In [ ]:
%load_ext tensorboard
# %reload_text tensorboard

logdir=".\\logs\\fit\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

# Callback function with early stopping to avodid overfit

class Callback_Configurations():
    
    MONITOR_METRIC = 'val_loss'
    MINIMUM_DELTA = 0.1
    PATIENCE = 100
    VERBOSE = 1
    MODE = 'min'
    
def callback(saved_model, model, logdir):
    
    weights_fname = os.path.join(os.getcwd(), 'model_one\\{}.h5'.format(saved_model))

    try:
        with open(os.path.join(os.getcwd(), 'model_one\\{}.json'.format(save_model)), 'r') as f:
            
            model_json = json.load(f)
        
            model = model_from_json(model_json)
        
            model.load_weights('{}').format(weights_fname)

    except:
        print('\nPre-trained weights not found. Fitting from start')
        pass

    monitor_metric = Callback_Configurations.MONITOR_METRIC
    
    callbacks = [
        tfmodel.EpochDots(),
        
        EarlyStopping(monitor=monitor_metric,
                      min_delta=Callback_Configurations.MINIMUM_DELTA,
                      patience=Callback_Configurations.PATIENCE,
                      verbose=Callback_Configurations.VERBOSE,
                      mode=Callback_Configurations.MODE,
                      restore_best_weights=True),

        ModelCheckpoint(filepath=weights_fname,
                        monitor=monitor_metric,
                        verbose=Callback_Configurations.VERBOSE,
                        save_best_only=True,
                        save_weights_only=True), #True, False
        
        tf.keras.callbacks.TensorBoard(logdir)
    ]
    return callbacks

# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

# Function to fit the keras multy input model

def fitting_model(method,
                  model,
                  x_train_seq_actors, 
                  x_train_seq_plot, 
                  x_train_seq_features,
                  x_train_seq_reviews,
                  x_test_seq_actors, 
                  x_test_seq_plot, 
                  x_test_seq_features,
                  x_test_seq_reviews,
                  y_train, 
                  y_test,
                  callbacks,
                  steps_per_epoch,
                  epoch,
                  verbose_fit,
                  batch_size_fit,
                  val_split):
    """
    Instead of using validation_data, I used the validation_split parameter
    """
    s = time()
    
    if method=="validation_split":
        
        fit_model = model.fit([x_train_seq_actors, x_train_seq_plot, x_train_seq_features, x_train_seq_reviews], y_train,
                              steps_per_epoch = steps_per_epoch,
                              epochs=epoch,
                              verbose=verbose_fit,
                              batch_size=batch_size_fit,
                              validation_split=val_split, # 90% for train and 10% for validation
                              #validation_data=([x_test_seq_actors, x_test_seq_plot, x_test_seq_features, x_test_seq_reviews], y_test),
                              callbacks=callbacks) #(callbacks)
    elif method=="validation_data":
        
        fit_model = model.fit([x_train_seq_actors, x_train_seq_plot, x_train_seq_features, x_train_seq_reviews], y_train,
                              steps_per_epoch = steps_per_epoch,
                              epochs=epoch,
                              verbose=verbose_fit,
                              batch_size=batch_size_fit,
                              validation_data=([x_test_seq_actors, x_test_seq_plot, x_test_seq_features, x_test_seq_reviews], y_test),
                              callbacks=callbacks) #(callbacks)

    duration = time() - s
    print("\nTraining time finished. Duration {} secs".format(duration))
    
    return fit_model

# For Kfold cross validation!
def fitting_model_kfold(model, 
                        x_seq_actors_train, 
                        x_seq_plot_train, 
                        x_seq_features_train,
                        x_seq_reviews_train,
                        x_seq_actors_test, 
                        x_seq_plot_test, 
                        x_seq_features_test,
                        x_seq_reviews_test,
                        y_train,
                        y_test,
                        callbacks,
                        steps_per_epoch,
                        epoch,
                        verbose_fit,
                        batch_size_fit):
    s = time()

    fit_model = model.fit([x_seq_actors_train, x_seq_plot_train, x_seq_features_train, x_seq_reviews_train], y_train,
                          steps_per_epoch = steps_per_epoch,
                          epochs=epoch,
                          verbose=verbose_fit,
                          batch_size=batch_size_fit,
                          validation_data=([x_seq_actors_test, x_seq_plot_test, x_seq_features_test, x_seq_reviews_test], y_test),
                          callbacks=callbacks) #(callbacks)

    duration = time() - s
    print("\nTraining time finished. Duration {} secs".format(duration))
    
    return fit_model

# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

def save_model(model, model_name):
    
    model_json = model.to_json()

    with open(os.path.join(os.getcwd(), "model_one\\{}.json".format(model_name)), "w") as json_file:
        json.dump(model_json, json_file)

    model.save_weights(os.path.join(os.getcwd(), "model_one\\{}.h5".format(model_name)))
    
    print("\nModel's weights are saved")
    
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

# function to plot the model metrics (deprecated)

def plot_model_metrics(fit_model):

    rcParams['figure.figsize'] = 12, 6

    plt.plot(fit_model.history['accuracy'] , 'g') # acc
    plt.plot(fit_model.history['val_accuracy'] , 'b') # val_acc
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.grid(True)
    plt.show()

    rcParams['figure.figsize'] = 12, 6

    plt.plot(fit_model.history['loss'] , 'g')
    plt.plot(fit_model.history['val_loss'] , 'b')
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.grid(True)
    plt.show()

# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

# alternative function to plot the model metrics (used)

def plot_keras_history(history): #where history =  model.fit()
    """
    
    :param history: 
    :return: 
    """
    # the history object gives the metrics keys. 
    # we will store the metrics keys that are from the training sesion.
    metrics_names = [key for key in history.history.keys() if not key.startswith('val_')]

    for i, metric in enumerate(metrics_names):
        
        # getting the training values
        metric_train_values = history.history.get(metric, [])
        
        # getting the validation values
        metric_val_values = history.history.get("val_{}".format(metric), [])

        # As loss always exists as a metric we use it to find the 
        epochs = range(1, len(metric_train_values) + 1)
        
        # leaving extra spaces to allign with the validation text
        training_text = "   Training {}: {:.5f}".format(metric,
                                                        metric_train_values[-1])

        # metric
        plt.figure(i, figsize=(12, 6))

        plt.plot(epochs,
                 metric_train_values,
                 'b',
                 label=training_text)
        
        # if we validation metric exists, then plot that as well
        if metric_val_values:
            validation_text = "Validation {}: {:.5f}".format(metric,
                                                             metric_val_values[-1])

            plt.plot(epochs,
                     metric_val_values,
                     'g',
                     label=validation_text)
        
        # add title, xlabel, ylabe, and legend
        plt.title('Model Metric: {}'.format(metric))
        plt.xlabel('Epochs')
        plt.ylabel(metric.title())
        plt.legend()
    
    fig1 = plt.gcf()
    #plt.savefig('multi-input-keras.png')
    plt.show()
    plt.draw()
    fig1.savefig(os.path.join(os.getcwd(), 'model_one\\multi_input_keras.png'), dpi=100)

# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

# Create function that will get as input a dataframe with the metrics (validation, accuracy) and create a plot per epoch
# Proposed modules: seaborn, plotly, matplolib.

In [ ]:
# Neural Network Structure

neural_network_parameters = {}
optimizer_parameters = {}
fit_parameters = {}

neural_network_parameters['embedding_dimension'] = 50 #Grid-search on that value
neural_network_parameters['pool_size'] = None
neural_network_parameters['padding'] = 'valid'
neural_network_parameters['batch_size'] = 128 #Grid-search on that value
neural_network_parameters['l2_regularization'] = 0.01 #Grid-search on that value
neural_network_parameters['dropout_rate'] = 0.0 #Grid-search on that value
neural_network_parameters['dense_activation'] = 'relu'
neural_network_parameters['output_activation'] = 'sigmoid' #softmax, sigmoid
neural_network_parameters['number_target_variables'] = len(y_train[0].tolist())

#It’s important to notice, that we use a sigmoid activation function with a multiclass output-layer. 
#The sigmoid gives us independent propabilities for each class. So DON’T use softmax here!

neural_network_parameters['model_loss'] = 'binary_crossentropy' #sparse_categorical_crossentropy, binary_crossentropy, categorical_crossentropy
neural_network_parameters['model_metric'] = 'accuracy' #sparse_categorical_accuracy, accuracy

#--------------------------------------------------------------------------------------

fit_parameters["steps_per_epoch"] = len(X_train_seq_features)//neural_network_parameters['batch_size']
fit_parameters["epoch"] = 150
fit_parameters["verbose_fit"] = 0
fit_parameters["batch_size_fit"] = 128 #Grid-search on that value

#---------------------------------------------------------------------------------------

# Optimizer: ADAM (version_1)

optimizer_parameters['adam_learning_rate'] = 0.001
optimizer_parameters['adam_beta_1'] = 0.9
optimizer_parameters['adam_beta_2'] = 0.999
optimizer_parameters['adam_amsgrad'] = False

def optimizer_adam_v1():
    
    return keras.optimizers.Adam(learning_rate=optimizer_parameters['adam_learning_rate'], 
                                 beta_1=optimizer_parameters['adam_beta_1'], 
                                 beta_2=optimizer_parameters['adam_beta_2'], 
                                 amsgrad=optimizer_parameters['adam_amsgrad'])
#---------------------------------------------------------------------------------------

# Optimizer: ADAM (version_2)

# lr_schedule_learning_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
# lr_schedule_decay_rate = [1, 2, 3, 4]

optimizer_parameters['steps_per_epoch'] = len(X_train_seq_features)//neural_network_parameters['batch_size']
# Steps per epoch...fine tune
optimizer_parameters['lr_schedule_learning_rate'] = 0.01 #Grid-search on that value
optimizer_parameters['lr_schedule_decay_steps'] = optimizer_parameters['steps_per_epoch']*1000
optimizer_parameters['lr_schedule_decay_rate'] = 1 #Grid-search on that value
optimizer_parameters['staircase'] = False

#STEPS_PER_EPOCH = len(X_train_seq_features)//neural_network_parameters['batch_size'] #(512 = BATCH SIZE)

lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
    optimizer_parameters['lr_schedule_learning_rate'],
    decay_steps=optimizer_parameters['lr_schedule_decay_steps'],
    decay_rate=optimizer_parameters['lr_schedule_decay_rate'],
    staircase=optimizer_parameters['staircase'])

def optimizer_adam_v2():
    
    return keras.optimizers.Adam(lr_schedule)
#---------------------------------------------------------------------------------------

# Optimizer: SDG (version 1)

optimizer_parameters['SGD_learning_rate'] = 0.01
optimizer_parameters['SGD_decay'] = 1e-6
optimizer_parameters['SGD_momentum'] = 0.9
optimizer_parameters['SGD_nesterov'] = True

def optimizer_SDG_v1():
    
    return keras.optimizers.SGD(lr=optimizer_parameters['SGD_learning_rate'],
                                decay=optimizer_parameters['SGD_decay'],
                                momentum=optimizer_parameters['SGD_momentum'],
                                nesterov=optimizer_parameters['SGD_nesterov'])

#---------------------------------------------------------------------------------------

In [ ]:
def keras_multy_classification_model_v6(maxlen_actors, 
                                        maxlen_plot, 
                                        maxlen_features, 
                                        maxlen_reviews,
                                        actors_max_words, 
                                        plot_max_words,
                                        features_max_words,
                                        reviews_max_words,
                                        optimizer_version = None):
    
    sentenceLength_actors = maxlen_actors
    vocab_size_frequent_words_actors = actors_max_words #20001, 12527, 11066, 16334
    
    sentenceLength_plot = maxlen_plot
    vocab_size_frequent_words_plot = plot_max_words #17501, 13295, 10910, 10084
    
    sentenceLength_features = maxlen_features
    vocab_size_frequent_words_features = features_max_words #20001, 17666, 14965, 14440
    
    sentenceLength_reviews = maxlen_reviews
    vocab_size_frequent_words_reviews = reviews_max_words #40001, 16100, 15405, 15251
    
    model = keras.Sequential(name='MultyInput_Keras_Classification_model')
    
    actors = keras.Input(shape=(sentenceLength_actors,), name='actors_input')
    plot = keras.Input(shape=(sentenceLength_plot,), name='plot_input')
    features = keras.Input(shape=(sentenceLength_features,), name='features_input')
    reviews = keras.Input(shape=(sentenceLength_reviews,), name='reviews_input')
    
    emb1 = layers.Embedding(input_dim = vocab_size_frequent_words_actors + 1,
                            output_dim = neural_network_parameters['embedding_dimension'],
                            embeddings_initializer = 'uniform',
                            mask_zero = True,
                            input_length = sentenceLength_actors,
                            name="actors_embedding_layer")(actors)
    
    encoded_layer1 = layers.GlobalMaxPooling1D(name="globalmaxpooling1")(emb1)
    
    emb2 = layers.Embedding(input_dim = vocab_size_frequent_words_plot + 1,
                            output_dim = neural_network_parameters['embedding_dimension'],
                            embeddings_initializer = 'uniform',
                            mask_zero = True,
                            input_length = sentenceLength_plot,
                            name="plot_embedding_layer")(plot)
    
    encoded_layer2 = layers.GlobalMaxPooling1D(name="globalmaxpooling2")(emb2)

    emb3 = layers.Embedding(input_dim = vocab_size_frequent_words_features + 1,
                            output_dim = neural_network_parameters['embedding_dimension'],
                            embeddings_initializer = 'uniform',
                            mask_zero = True,
                            input_length = sentenceLength_features,
                            name="features_embedding_layer")(features)
    
    encoded_layer3 = layers.GlobalMaxPooling1D(name="globalmaxpooling3")(emb3)
    
    emb4 = layers.Embedding(input_dim = vocab_size_frequent_words_reviews + 1,
                            output_dim = neural_network_parameters['embedding_dimension'],
                            embeddings_initializer = 'uniform',
                            mask_zero = True,
                            input_length = sentenceLength_reviews,
                            name="reviews_embedding_layer")(reviews)
    
    encoded_layer4 = layers.GlobalMaxPooling1D(name="globalmaxpooling4")(emb4) #2D for images
    
    merged = layers.concatenate([encoded_layer1, encoded_layer2, encoded_layer3, encoded_layer4], axis=-1)

    dense_layer_1 = layers.Dense(neural_network_parameters['batch_size'], 
                                 kernel_regularizer=regularizers.l2(neural_network_parameters['l2_regularization']),
                                 activation=neural_network_parameters['dense_activation'],
                                 name="1st_dense_layer")(merged)
    layers.Dropout(neural_network_parameters['dropout_rate'])(dense_layer_1)
    
    output_layer = layers.Dense(neural_network_parameters['number_target_variables'], activation=neural_network_parameters['output_activation'],
                                name='output_layer')(dense_layer_1)
    
    model = keras.Model(inputs=[actors, plot, features, reviews], outputs=output_layer)
    
    print(model.output_shape)

    print(model.summary())
    
    # Version_1 of Adam
    if optimizer_version is None:
        
        optimizer = optimizer_adam_v2()
    
    model.compile(optimizer=optimizer,
                  loss=neural_network_parameters['model_loss'],
                  metrics=[neural_network_parameters['model_metric']])
    
    plot_model(model, to_file=os.path.join(os.getcwd(), 'model_one\\structure_multy_input_keras_model.png'))
    
    return model

#### Prior to fitting the model: 

* X_train, X_test should have the form of an array with sequence of numbers.
* y_train, y_test should have the form of a multi-hot encoded dataframe.

<b> General observations: </b>

* Reducing batch size can produce a better model (I should grid search on batch size).
* Reducing the general number of parameters can produce better results.
* Removing the second dense layer improved the results.
* Removing regularization also affected the results.

In [ ]:
# RUN MLFLOW program

model_repository = {}

model_repository['Multiple Input Model'] = keras_multy_classification_model_v6(X_train_seq_actors.shape[1], 
                                                                               X_train_seq_plot.shape[1], 
                                                                               X_train_seq_features.shape[1], 
                                                                               X_train_seq_reviews.shape[1],
                                                                               len(actors_tokenizer.word_index),
                                                                               len(plot_tokenizer.word_index),
                                                                               len(features_tokenizer.word_index),
                                                                               len(reviews_tokenizer.word_index),
                                                                               optimizer_version = None)
mlflow.set_experiment("/multi_input_keras_model")
with mlflow.start_run():
    
    model_directory = os.path.join(os.getcwd(), "model_one")
    logdir = ".\\logs_test\\fit\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    #callbacks = callback("model_multy_input", model_repository['Multiple Input Model'], logdir)

    model_history = {}
    model_history['experiment'] = fit_keras_multy_input(model_repository['Multiple Input Model'],
                                                        X_train_seq_actors, #input_1
                                                        X_train_seq_plot, #input_2
                                                        X_train_seq_features, #input_3
                                                        X_train_seq_reviews, #input4
                                                        X_test_seq_actors,
                                                        X_test_seq_plot,
                                                        X_test_seq_features,
                                                        X_test_seq_reviews,
                                                        y_train, #output
                                                        y_test,
                                                        callback("multi_input_keras_model", model_repository['Multiple Input Model'], logdir), #callback function
                                                        fit_parameters["steps_per_epoch"],
                                                        fit_parameters["epoch"],
                                                        fit_parameters["verbose_fit"],
                                                        fit_parameters["batch_size_fit"])

    hist = pd.DataFrame(model_history['experiment'].history)
    hist['epoch'] = model_history['experiment'].epoch
    hist['epoch']+= 1
    hist.index += 1
    print("\nTable of training the keras text classification model\n")
    print(tabulate(hist, headers='keys', tablefmt='psql'))
    
    hist.to_pickle(os.path.join(os.getcwd(), "model_one\\metrics_histogram_multi_input_keras.pkl"))
    
    #save the model
    save_model(model_repository['Multiple Input Model'], "multi_input_keras_model")
    
    #plot the model's accuracy & loss
    plot_keras_history(model_history['experiment'])
    
    # - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
    
    # evaluate the model
    model_evaluation = model_repository['Multiple Input Model'].evaluate([X_test_seq_actors, X_test_seq_plot, X_test_seq_features, X_test_seq_reviews], 
                                                                         y_test,
                                                                         batch_size=fit_parameters["batch_size_fit"],
                                                                         verbose=2)
    print('\nTest Score:', model_evaluation[0])

    print('\nTest Accuracy:', model_evaluation[1])
    
    #neural_model params
    mlflow.log_param("embedding_dimension", neural_network_parameters['embedding_dimension'] )
    mlflow.log_param("pool_size", neural_network_parameters['pool_size'])
    mlflow.log_param("padding", neural_network_parameters['padding'])
    mlflow.log_param("batch_size", neural_network_parameters['batch_size'])
    mlflow.log_param("l2_regularization", neural_network_parameters['l2_regularization'])
    mlflow.log_param("dropout_rate", neural_network_parameters['dropout_rate'])
    mlflow.log_param("dense_activation", neural_network_parameters['dense_activation'])
    mlflow.log_param("output_activation",neural_network_parameters['output_activation'])
    mlflow.log_param("model_loss",neural_network_parameters['model_loss']) #takes any data type
    mlflow.log_param("model_metric",neural_network_parameters['model_metric'])
    
    #optimizer params
    mlflow.log_param("lr_schedule_learning_rate",optimizer_parameters['lr_schedule_learning_rate'])
    mlflow.log_param("lr_schedule_decay_steps",optimizer_parameters['lr_schedule_decay_steps'])
    mlflow.log_param("lr_schedule_decay_rate",optimizer_parameters['lr_schedule_decay_rate'])
    mlflow.log_param("adam_amsgrad",optimizer_parameters['staircase'])
    
    #fit_model params
    mlflow.log_param("steps_per_epoch",fit_parameters['steps_per_epoch'])
    mlflow.log_param("fit_epoch",fit_parameters['epoch'])
    mlflow.log_param("verbose_fit",fit_parameters['verbose_fit'])
    mlflow.log_param("batch_size_fit",fit_parameters['batch_size_fit']) #in generl batch_size_fit = neurons batch size
    
    #logging the model metrics
    mlflow.log_metric("model_validation_loss",model_evaluation[0]) #take only floats/integers
    mlflow.log_metric("model_validation_accuracy",model_evaluation[1])
    
    #mlflow.keras.save_model(model_repository['Multiple Input Model'], model_directory) -> deprecated
    mlflow.keras.log_model(model_repository['Multiple Input Model'], "multi_input_keras_model")

In [ ]:
# RUN MLFLOW program with KFold Cross Validation
from sklearn.model_selection import StratifiedKFold, KFold
seed=123
mlflow.set_experiment("multi_input_keras_model")
with mlflow.start_run():
    
    model_repository = {}
    model_repository['Multiple Input Model'] = keras_multy_classification_model_v6(X_seq_actors.shape[1], 
                                                                                   X_seq_plot.shape[1], 
                                                                                   X_seq_features.shape[1], 
                                                                                   X_seq_reviews.shape[1], 
                                                                                   optimizer_version = None)
    
    model_directory = "C:\\Users\\spano\\Desktop\\GitHub-Thesis\\models_text_classification\\model_one"
    
    logdir = ".\\logs_test\\fit\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

    callbacks = callback("model_multy_input", model_repository['Multiple Input Model'], logdir)
    
    kfold = KFold(n_splits=5, shuffle=True, random_state=seed)
    cvscores = []
    for train, test in kfold.split(X_seq_actors, y):
    
        model_history = {}
        model_history['experiment'] = fit_keras_multy_input(model_repository['Multiple Input Model'], 
                                                            X_seq_actors[train], #input_1
                                                            X_seq_plot[train], #input_2
                                                            X_seq_features[train], #input_3
                                                            X_seq_reviews[train], #input4
                                                            X_seq_actors[test], 
                                                            X_seq_plot[test], 
                                                            X_seq_features[test],
                                                            X_seq_reviews[test],
                                                            y[train], #output
                                                            y[test], 
                                                            callbacks, #callback function
                                                            fit_parameters["steps_per_epoch"],
                                                            fit_parameters["epoch"],
                                                            fit_parameters["verbose_fit"],
                                                            fit_parameters["batch_size_fit"])

    hist = pd.DataFrame(model_history['experiment'].history)
    hist['epoch'] = model_history['experiment'].epoch
    hist['epoch']+= 1
    hist.index += 1
    print("\nTable of training the keras text classification model\n")
    print(tabulate(hist, headers='keys', tablefmt='psql'))
    
    hist.to_pickle(".\\model_one\\metrics_histogram_multi_input_keras.pkl")
    
    save_model(model_repository['Multiple Input Model'], "model_multy_input")
    
    #version_1 of plot model
    #plot_model_metrics(model_history['experiment'])
    
    #version_2 of plot model
    plot_keras_history(model_history['experiment'])
    
    # - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

    model_evaluation = model_repository['Multiple Input Model'].evaluate([X_seq_actors[test], X_seq_plot[test], X_seq_features[test], X_seq_reviews[test]], 
                                                                         y[test],
                                                                         batch_size=fit_parameters["batch_size_fit"],
                                                                         verbose=2)
    print('\nTest Score:', model_evaluation[0])

    print('\nTest Accuracy:', model_evaluation[1])
    
    print("%.2f%% (+/- %.2f%%)" % (np.mean(model_evaluation), np.std(model_evaluation)))
    
    #neural_model params
    mlflow.log_param("embedding_dimension", neural_network_parameters['embedding_dimension'] )
    mlflow.log_param("pool_size", neural_network_parameters['pool_size'])
    mlflow.log_param("padding", neural_network_parameters['padding'])
    mlflow.log_param("batch_size", neural_network_parameters['batch_size'])
    mlflow.log_param("l2_regularization", neural_network_parameters['l2_regularization'])
    mlflow.log_param("dropout_rate", neural_network_parameters['dropout_rate'])
    mlflow.log_param("dense_activation", neural_network_parameters['dense_activation'])
    mlflow.log_param("output_activation",neural_network_parameters['output_activation'])
    mlflow.log_param("model_loss",neural_network_parameters['model_loss']) #takes any data type
    mlflow.log_param("model_metric",neural_network_parameters['model_metric'])
    
    #optimizer params
    mlflow.log_param("lr_schedule_learning_rate",optimizer_parameters['lr_schedule_learning_rate'])
    mlflow.log_param("lr_schedule_decay_steps",optimizer_parameters['lr_schedule_decay_steps'])
    mlflow.log_param("lr_schedule_decay_rate",optimizer_parameters['lr_schedule_decay_rate'])
    mlflow.log_param("adam_amsgrad",optimizer_parameters['staircase'])
    
    #fit_model params
    mlflow.log_param("steps_per_epoch",fit_parameters['steps_per_epoch'])
    mlflow.log_param("fit_epoch",fit_parameters['epoch'])
    mlflow.log_param("verbose_fit",fit_parameters['verbose_fit'])
    mlflow.log_param("batch_size_fit",fit_parameters['batch_size_fit']) #in generl batch_size_fit = neurons batch size
    
    #logging the model metrics
    mlflow.log_metric("model_validation_loss",model_evaluation[0]) #take only floats/integers
    mlflow.log_metric("model_validation_accuracy",model_evaluation[1])
    
    #mlflow.keras.save_model(model_repository['Multiple Input Model'], model_directory) -> deprecated
    mlflow.keras.log_model(model_repository['Multiple Input Model'], "multi-input-keras-model")

    #mlflow.tensorflow.log_model(tf_saved_model_dir = "C:\\Users\\spano\\Desktop\\GitHub-Thesis\\models_text_classification\\model_one\\model_multy_input.h5")
#   mlflow.tensorflow.save_model(model_repository['Multiple Input Model'], model_directory)

#### - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

<b> Model prediction </b>

A good practice is to save the model predictions, to reproduce the classification report and compare it to other models

In [ ]:
with open(os.path.join(os.getcwd(), "pickled_data_per_part\\genres_list_16022020.pkl"), 'rb') as handle:
    genres_list = pickle.load(handle)

In [ ]:
# Use to yield probability distribution over the categories
y_test_pred_probs = model_repository['Multiple Input Model'].predict([X_test_seq_actors, X_test_seq_plot, X_test_seq_features, X_test_seq_reviews])
y_test_pred_probs[0]

# y_predicted probabilities for each class

In [ ]:
y_test_predictions = (y_test_pred_probs>0.5).astype(int)

In [ ]:
y_test_predictions[4314]
# y_predicted genre if equals to 1 (predicted genres of the movie)

In [ ]:
y_test[4314]
# the real genres of the movies

In [ ]:
np.save(os.path.join(os.getcwd(), "model_one//y_predictions_multi_input_keras_07032020"), y_test_predictions)
np.save(os.path.join(os.getcwd(), "model_one//y_true_multi_input_keras_07032020"), y_test)

In [ ]:
# Classification Report

classification_table= classification_report(y_true=y_test, y_pred=y_test_predictions)

print("Classification report\n" + str(classification_table))

# Hamming Loss
print("Hamming loss: ", hamming_loss(y_test, y_test_predictions))

# Zero_one loss
print("Zero one loss: ", zero_one_loss(y_test, y_test_predictions, normalize=False))

In [ ]:
conf_mat=confusion_matrix(y_test.argmax(axis=1), y_test_predictions.argmax(axis=1))

conf_matrix=pd.DataFrame(conf_mat,
             columns=genres_list,
             index=genres_list)
conf_matrix.to_pickle(os.path.join(os.getcwd(), "model_one\\confusion_matrix_07032020.pkl"))
conf_matrix

#### - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

#### Metrics Dataframe

In [ ]:
df_scores = pd.DataFrame({'Keras Model':pd.Series("Multy Input Keras", dtype='str'),
                         'Test Loss':pd.Series([model_evaluation[0]], dtype='float'),
                         'Test Accuracy':pd.Series([model_evaluation[1]], dtype='float'),
                         'Hamming Loss':pd.Series([hamming_loss(y_test, y_test_predictions)], dtype='float'),
                         'Zero_one Loss':pd.Series([zero_one_loss(y_test, y_test_predictions, normalize=False)], dtype='float'),
                         'F1_score':pd.Series([f1_score(y_test, y_test_predictions, average="micro")], dtype='float')})

df_scores.to_pickle(os.path.join(os.getcwd(), "model_one\\df_metrics_multy_input_keras_08032020.pkl"))

df_scores

#### Predicted vs Actual Genre Tags

In [ ]:
def predict_genre_tags(indx, model, genres_list):
        
    test_sequence_actors = X_test_seq_actors[indx:indx+1]
    
    test_sequence_plot = X_test_seq_plot[indx:indx+1]
    
    test_sequence_features = X_test_seq_features[indx:indx+1]
    
    test_sequence_reviews = X_test_seq_reviews[indx:indx+1]
    
    text_prediction = model.predict([test_sequence_actors, test_sequence_plot, test_sequence_features, test_sequence_reviews])
    
    [float(i) for i in text_prediction[0]]
    
    tag_probabilities = text_prediction[0][np.argsort(text_prediction[0])[-3:]]
    
    indexes = np.argsort(text_prediction[0])[::-1][:3]

    predicted_tags = []
    
    for i, tag in enumerate(genres_list):
        if i in indexes:
            predicted_tags.append(genres_list[i])
    
    return predicted_tags

In [ ]:
random_numbers = random.sample(range(1, y_test.shape[0]), 5)

save_index_of_numbers = random_numbers

print("Randomly saved numbers to make predictions: {}".format(save_index_of_numbers))

In [ ]:
X_test=pd.read_pickle(os.path.join(os.getcwd(), "80-20 split_non-balanced\\x_test_06032020.pkl"))

In [ ]:
df_predictions = pd.DataFrame({'Movie Title':pd.Series([X_test['title'].iloc[save_index_of_numbers[0]]], dtype='str'),
                               'Predicted Genre tags':pd.Series([predict_genre_tags(save_index_of_numbers[0], model_repository['Multiple Input Model'], genres_list)], dtype='str'),
                               'Real Genre tags':pd.Series([X_test['reduced_genres'].iloc[save_index_of_numbers[0]]], dtype='str')})

for i in range(len(save_index_of_numbers)):

    df_predictions = df_predictions.append({'Movie Title':X_test['title'].iloc[save_index_of_numbers[i]], 
                                            'Predicted Genre tags':predict_genre_tags(save_index_of_numbers[i], model_repository['Multiple Input Model'], genres_list),
                                            'Real Genre tags':X_test['reduced_genres'].iloc[save_index_of_numbers[i]]} , ignore_index=True)

df_predictions = df_predictions.drop(df_predictions.index[0])
df_predictions.to_pickle("model_one\\model_one_df_predictions_07032020.pkl")
df_predictions

<b>- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -  </b>